In [1]:
import requests
import os
key = os.environ["ecos_api_key"]

In [ ]:
!pip install xmltodict

In [2]:
import xmltodict
import json
import pandas as pd
api = lambda url: json.loads(json.dumps(xmltodict.parse(requests.get(url).text)))

### 서비스 통계 목록
* 통계 목록 조회 api
* [API 명세](https://ecos.bok.or.kr/jsp/openapi/OpenApiController.jsp?t=guideServiceDtl&apiCode=OA-1020&menuGroup=MENU000004)

In [ ]:
url = f'http://ecos.bok.or.kr/api/StatisticTableList/{key}/xml/kr/1/100/' # /start/end/
result = api(url)
result

In [ ]:
tables = result["StatisticTableList"]["row"]
tables[0] 

#### `cycle == None`: 조회할 수 없는 대분류 코드

In [ ]:
tables[1]

### 통계 세부항목 목록
* 통계 세부항목 조회 api
* [API 명세](https://ecos.bok.or.kr/jsp/openapi/OpenApiController.jsp?t=guideServiceDtl&apiCode=OA-1030&menuGroup=MENU000004)

In [ ]:
url = f'http://ecos.bok.or.kr/api/StatisticItemList/{key}/xml/kr/1/100/050Y002/'
result = api(url)
result

In [ ]:
result['StatisticItemList']['row'][0]

### 통계 조회 조건 설정
* 통계값 조회 api
* [API 명세](https://ecos.bok.or.kr/jsp/openapi/OpenApiController.jsp?t=guideServiceDtl&apiCode=OA-1040&menuGroup=MENU000004)

In [ ]:
url = f'http://ecos.bok.or.kr/api/StatisticSearch/{key}/xml/kr/1/100/050Y002/HM/2008021/2008022/1020000/'
result = api(url)
len(result["StatisticSearch"]['row']) # 조건

In [ ]:
url = f'http://ecos.bok.or.kr/api/StatisticSearch/{key}/xml/kr/1/100/050Y002/HM/2008021/2008022/'
result = api(url)
len(result["StatisticSearch"]['row']) # 전수

In [ ]:
first_row = result['StatisticSearch']['row'][0]
print(first_row["STAT_CODE"], first_row["STAT_NAME"], result["StatisticSearch"]["list_total_count"])
pd.DataFrame(result['StatisticSearch']['row'])

### 데이터 수급
1. 서비스 통계 목록
  * 전체 목록 받아서 `if cycle` 인 것만 실제 통계
1. 통계 세부항목 목록
  * 1번 항목에서 받아온 테이블 옵션 `ITEM_CODE` 목록 확인 가능
  * `START_TIME`, `END_TIME` 확인
1. 통계 조회 조건 설정
  * 1번 항목에서 받아온 테이블 조회
  * `ITEM_CODE` 설정하면 필터링, 안하면 전수 데이터 리턴

In [3]:
url = f'http://ecos.bok.or.kr/api/StatisticTableList/{key}/xml/kr/1/1000/' # /start/end/
result = api(url)
result

{'StatisticTableList': {'list_total_count': '880',
  'row': [{'P_STAT_CODE': '*',
    'STAT_CODE': '000Y005',
    'STAT_NAME': '1.통화 및 유동성지표',
    'CYCLE': None,
    'SRCH_YN': 'N',
    'ORG_NAME': None},
   {'P_STAT_CODE': '000Y005',
    'STAT_CODE': '010Y002',
    'STAT_NAME': '1.1.주요 통화금융지표',
    'CYCLE': 'MM',
    'SRCH_YN': 'Y',
    'ORG_NAME': None},
   {'P_STAT_CODE': '000Y005',
    'STAT_CODE': '000Y074',
    'STAT_NAME': '1.2.본원통화',
    'CYCLE': None,
    'SRCH_YN': 'N',
    'ORG_NAME': None},
   {'P_STAT_CODE': '000Y074',
    'STAT_CODE': '000Y701',
    'STAT_NAME': '1.2.1 본원통화 구성내역(말잔)',
    'CYCLE': None,
    'SRCH_YN': 'N',
    'ORG_NAME': None},
   {'P_STAT_CODE': '000Y701',
    'STAT_CODE': '054Y407',
    'STAT_NAME': '1.2.1.1 본원통화 구성내역(말잔, 계절조정계열)',
    'CYCLE': 'MM',
    'SRCH_YN': 'Y',
    'ORG_NAME': None},
   {'P_STAT_CODE': '000Y701',
    'STAT_CODE': '054Y007',
    'STAT_NAME': '1.2.1.2 본원통화 구성내역(말잔,원계열)',
    'CYCLE': 'MM',
    'SRCH_YN': 'Y',
    'ORG_NAME': Non

In [4]:
tables = pd.DataFrame(result["StatisticTableList"]["row"])
tables = tables.loc[tables["SRCH_YN"]=="Y"]
tables

,P_STAT_CODE,STAT_CODE,STAT_NAME,CYCLE,SRCH_YN,ORG_NAME
1,000Y005,010Y002,1.1.주요 통화금융지표,MM,Y,None
4,000Y701,054Y407,"1.2.1.1 본원통화 구성내역(말잔, 계절조정계열)",MM,Y,None
5,000Y701,054Y007,"1.2.1.2 본원통화 구성내역(말잔,원계열)",MM,Y,None
7,000Y702,054Y408,"1.2.2.1 본원통화 구성내역(평잔, 계절조정계열)",MM,Y,None
8,000Y702,054Y008,"1.2.2.2 본원통화 구성내역(평잔,원계열)",MM,Y,None
...,...,...,...,...,...,...
874,000Y178,I10Y002,18.1.6.2 주가지수,MM,Y,None
875,000Y178,I10Y004,18.1.6.3 국제상품가격,MM,Y,None
877,000Y111,034Y003,18.2.1 남북한 주요지표 비교(총량),YY,Y,None
878,000Y111,034Y002,18.2.2 남북한 주요지표 비교(한국/북한 배율),YY,Y,None


In [ ]:
import os
from tqdm.notebook import trange
files = set(os.listdir('./data/'))
for i in trange(len(tables), desc="progressing"):
    table = tables.iloc[i]
    name = table["STAT_NAME"][:30].replace('/', ',') + ".xlsx"
    if name in files:
        continue
    code, cycle = table["STAT_CODE"], table["CYCLE"]
    url = f'http://ecos.bok.or.kr/api/StatisticItemList/{key}/xml/kr/1/1/{code}/'
    try:
        result = api(url)["StatisticItemList"]["row"]
    except:
        print(url, "\n", api(url))
        continue
    start, end = result["START_TIME"], result["END_TIME"]
    url = f'http://ecos.bok.or.kr/api/StatisticSearch/{key}/xml/kr/1/1/{code}/{cycle}/{start}/{end}/' # total row 파악
    try:
        count = int(api(url)['StatisticSearch']['list_total_count'])
    except:
        print(url, "\n", api(url))
        continue
    if count <= 100000:
        url = f'http://ecos.bok.or.kr/api/StatisticSearch/{key}/xml/kr/1/{count}/{code}/{cycle}/{start}/{end}/' # select *
        result = api(url)['StatisticSearch']['row']
    else:
        si, ei = 1, 10000
        result = []
        while ei <= count:
            url = f'http://ecos.bok.or.kr/api/StatisticSearch/{key}/xml/kr/{si}/{ei}/{code}/{cycle}/{start}/{end}/'
            result += api(url)['StatisticSearch']['row']
            si += 10000
            ei += 10000
        url = f'http://ecos.bok.or.kr/api/StatisticSearch/{key}/xml/kr/{si}/{count}/{code}/{cycle}/{start}/{end}/'
        result += api(url)['StatisticSearch']['row']
    df = pd.DataFrame(result)
    df.to_excel(f'./data/{name}')

progressing:   0%|          | 0/699 [00:00<?, ?it/s]

In [11]:
len(dfs)

282

In [12]:
print(set([" ".join(df.columns) for df in dfs])) # 형식이 동일~
len(set([df.iloc[0, 1] for df in dfs])) # 중복된 거 체크

{'STAT_CODE STAT_NAME ITEM_CODE1 ITEM_NAME1 ITEM_CODE2 ITEM_NAME2 ITEM_CODE3 ITEM_NAME3 UNIT_NAME TIME DATA_VALUE'}


282

In [13]:
pop_list = []
code_set = set()
for i in range(len(dfs)):
    code = dfs[i].iloc[1, 0]
    if code in code_set:
        pop_list.append(i)
    else:
        code_set.add(code)
print(pop_list) # 중복 row

[]


In [ ]:
dfs.pop(452)  # 중복값 삭제
dfs.pop(452)

In [ ]:
!pip install xlsxwriter

In [33]:
import pandas as pd
files = set(os.listdir('data/'))
writer = pd.ExcelWriter('data/3.xlsx', engine='xlsxwriter')
for df in dfs:
    if df.iloc[0,1].split(".")[0] == "3":
        df.to_excel(writer, sheet_name=df.iloc[0,1][:30])
writer.save()
writer.close()

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12


In [14]:
for i in trange(len(tables), desc="progressing"):
#     if i < 45: # 중간에 끊기는 경우..
#         continue
    table = tables.iloc[i]
    name = table["STAT_NAME"][:30]
    print(name)

progressing:   0%|          | 0/699 [00:00<?, ?it/s]

1.1.주요 통화금융지표
1.2.1.1 본원통화 구성내역(말잔, 계절조정계열)
1.2.1.2 본원통화 구성내역(말잔,원계열)
1.2.2.1 본원통화 구성내역(평잔, 계절조정계열)
1.2.2.2 본원통화 구성내역(평잔,원계열)
1.3.1.1 M2 상품별구성내역(말잔, 계절조정계열)
1.3.1.2 M2 상품별구성내역(말잔,원계열)
1.3.2.1 M2 상품별구성내역(평잔, 계절조정계열)
1.3.2.2 M2 상품별구성내역(평잔,원계열)
1.3.3.1 M2 경제주체별 보유현황(말잔, 계절조정
1.3.3.2 M2 경제주체별 보유현황(말잔, 원계열)
1.3.4.1 M2 경제주체별 보유현황(평잔, 계절조정
1.3.4.2 M2 경제주체별 보유현황(평잔, 원계열)
1.3.5 M2 기관별구성내역(말잔)
1.3.6 M2 기관별구성내역(평잔)
1.4.예금취급기관 개관표  (종합표,말잔)
1.4.1 중앙은행 개관표(말잔)
1.4.2 기타예금취급기관 개관표(말잔)
1.5.1.1 Lf 상품별구성내역(말잔, 계절조정계열)
1.5.1.2 Lf 상품별 구성내역(말잔,원계열)
1.5.2.1 Lf 상품별구성내역(평잔, 계절조정계열)
1.5.2.2 Lf 상품별 구성내역(평잔,원계열)
1.5.3 Lf 기관별 구성내역(말잔)
1.5.4 Lf 기관별 구성내역(평잔)
1.6.1 L(광의유동성) 구성내역(말잔, 계절조정계열
1.6.2 L(광의유동성) 구성  내역(말잔,원계열)
2.1.한국은행 주요계정  (말잔)
2.2.화폐발행잔액
2.3.예금은행 지급준비액  (평잔)
2.4.예금은행 지급준비액  (평잔,신,2008.2월상
2.5.한국은행 원화대출금(말잔)
2.6.한국은행 기준금리 및 여수신금리
3.1.1 예금은행 종별예금(말잔)
3.1.2 예금은행 종별예금(평잔)
3.1.3 예금은행 예금주별예금(말잔)
3.1.4 예금은행 기간별 정기예금(말잔)
3.1.5 예금은행 총수신(말잔)
3.1.6 예금은행 총수신(평잔)
3.2.예금은행 자금별  대출금(말잔)
3.3.1.1 산업별대출금(예금취급기관)
3.3.1.2 시

'/'